In [1]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [ ]:
# api_end = "https://api.scryfall.com" # using tapped out api instaed. 
mtg_whole = pd.read_csv("mtg_modern_whole.csv")

mtg_whole.dtypes

In [ ]:
name_, id_= [],[] # this code pulls unique names and id's of cards for removing reprints.
for i,el in enumerate(mtg_whole['name']):
    if el not in name_:
        name_.append(el)
        id_.append(mtg_whole['id'][i])
len(id_)

In [ ]:
# removes reprints from data
mtg_whole = mtg_whole.query(f"id == {id_}")
# writes the cleanned dataframe to a csv. .(always save work!!!)
mtg_whole.to_csv('mtg_modern_clean.csv')

In [2]:
mtg_modern = pd.read_csv('mtg_modern_clean.csv')

/home/ed/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,15,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
mtg_modern['name'].where(mtg_modern['text'].isna()).dropna(how='all').shape # cards with no rules text. 

(314,)

In [4]:
# pulls only usable data for clustering.
mtg_whole_2 = mtg_modern[['name', 'names', 'power', 'toughness', 'text', 'type', 'types', 'subtypes', 'supertypes', 'rarity', 'colorIdentity', 'colors', 'convertedManaCost', 'manaCost', 'faceConvertedManaCost',  'side', ]] # filters that would matter.
#'id', 'borderColor', 'enhrecRank', 'tcgplayerProductId', 'uuid', 'watermark', 'variations', 'originalText', 'originalType', 'scryfallIllustrationId', 'scryfallOracleId'])
mtg_whole_2.head(3)

,name,names,power,toughness,text,type,types,subtypes,supertypes,rarity,colorIdentity,colors,convertedManaCost,manaCost,faceConvertedManaCost,side
0,Abundance,NaN,NaN,NaN,"If you would draw a card, you may instead choo...",Enchantment,Enchantment,NaN,NaN,rare,G,G,4.0,{2}{G}{G},NaN,NaN
1,Academy Researchers,NaN,2,2,When Academy Researchers enters the battlefiel...,Creature — Human Wizard,Creature,"Human,Wizard",NaN,uncommon,U,U,3.0,{1}{U}{U},NaN,NaN
2,Adarkar Wastes,NaN,NaN,NaN,{T}: Add {C}.\n{T}: Add {W} or {U}. Adarkar Wa...,Land,Land,NaN,NaN,rare,"U,W",NaN,0.0,NaN,NaN,NaN


In [5]:
names = ['Green','Blue','Black','Red','White']
start_ = ['G','U','B','R','W']

for i,el in enumerate(names): # creates columns with cost of colors
    mtg_whole_2[f'manaCost_{el}'] = mtg_whole_2['manaCost'].apply(lambda x: str(x).count(f'{start_[i]}'))
# bellow gennerates cost of colorless. 
mtg_whole_2['manaCost_colorless'] = mtg_whole_2['convertedManaCost'].astype('int') - sum([mtg_whole_2['manaCost_Green'],mtg_whole_2['manaCost_Red'],mtg_whole_2['manaCost_Black'],mtg_whole_2['manaCost_Blue'], mtg_whole_2['manaCost_White']])

for i,el in enumerate(names):
    # identityVV
    mtg_whole_2[f'colorIdentity_{el}'] = mtg_whole_2['colorIdentity'].apply(lambda x: str(x).count(f'{start_[i]}'))
mtg_whole_2['colorIdentity_colorless'] = [1 if x == True else 0 for x in mtg_whole_2['colorIdentity'].isna()]

mtg_whole_2.drop(columns = ['manaCost','colors'], inplace = True)
mtg_whole_2.head(2)

/home/ed/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/ed/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/ed/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

,name,names,power,toughness,text,type,types,subtypes,supertypes,rarity,colorIdentity,convertedManaCost,faceConvertedManaCost,side,manaCost_Green,manaCost_Blue,manaCost_Black,manaCost_Red,manaCost_White,manaCost_colorless,colorIdentity_Green,colorIdentity_Blue,colorIdentity_Black,colorIdentity_Red,colorIdentity_White,colorIdentity_colorless
0,Abundance,NaN,NaN,NaN,"If you would draw a card, you may instead choo...",Enchantment,Enchantment,NaN,NaN,rare,G,4.0,NaN,NaN,2,0,0,0,0,2,1,0,0,0,0,0
1,Academy Researchers,NaN,2,2,When Academy Researchers enters the battlefiel...,Creature — Human Wizard,Creature,"Human,Wizard",NaN,uncommon,U,3.0,NaN,NaN,0,2,0,0,0,1,0,1,0,0,0,0


In [6]:
# separates data by distinct card type for clustering within these.
creatures = mtg_whole_2.where(mtg_whole_2['types'].str.endswith('Creature')).dropna(how='all') # add artifact/enchanntment creatures.
spells = mtg_whole_2.where(mtg_whole_2['types'].str.endswith(('Sorcery', 'Instant'))).drop(columns = [ 'power', 'toughness']).dropna(how='all') # add trybal spells.

artifacts = mtg_whole_2.where(mtg_whole_2['types'].str.endswith('Artifact')).drop(columns = ['power', 'toughness']).dropna(how='all') # filter out artefact lands

enchantments = mtg_whole_2.where(mtg_whole_2['types'].str.endswith('Enchantment' )).drop(columns = [ 'power', 'toughness']).dropna(how='all')

planeswalkers = mtg_whole_2.where(mtg_whole_2['types'].str.endswith('Planeswalker')).drop(columns = [ 'power', 'toughness']).dropna(how='all')

In [7]:
# mtg_whole_2['types'].value_counts()
# spells['types'].value_counts()
comanders = creatures.where(creatures['type'].str.startswith('Legendary Creature')).dropna(how = 'all')

In [8]:
comanders.count()

name                       869
names                       36
power                      869
toughness                  869
text                       856
type                       869
types                      869
subtypes                   869
supertypes                 869
rarity                     869
colorIdentity              863
convertedManaCost          869
faceConvertedManaCost       12
side                        36
manaCost_Green             869
manaCost_Blue              869
manaCost_Black             869
manaCost_Red               869
manaCost_White             869
manaCost_colorless         869
colorIdentity_Green        869
colorIdentity_Blue         869
colorIdentity_Black        869
colorIdentity_Red          869
colorIdentity_White        869
colorIdentity_colorless    869
dtype: int64